# Neural Networks
In this assignment you will train a neural network to predict the skill of a surgeon performing a fundamental task. The JHU-ISI Gesture and Skill Assessment Working Set [1] [2] is an open-source dataset collected at Johns Hopkins of trainee and expert surgeons performing basic tasks like knot tying, suturing and needle passing with the da Vinci robot[3]. <br>
<br>

In suturing, surgeons 'stitch up a wound' by passing a needle from one side of the tissue to the other. Check out the included videos to watch the surgeons perform a practice suturing task.
<br> 

In this notebook you will be predicting the skill of the operating surgeon from the suturing kinematics and videos using a <b>2-layer, linear network</b>. 

### References 
[1] Gao, Yixin, et al. "Jhu-isi gesture and skill assessment working set (jigsaws): A surgical activity dataset for human motion modeling." MICCAI Workshop: M2CAI. Vol. 3. 2014. <br>
[2] https://cirl.lcsr.jhu.edu/research/hmm/datasets/jigsaws_release/ <br> 
[3] https://www.intuitive.com/en-us/products-and-services/da-vinci?gclid=Cj0KCQiAwP3yBRCkARIsAABGiPo79mPGJFNXWFc8tEpuRgU_s61N1zsmGR552MFbJ5C_LW12gXlG8AoaAmlIEALw_wcB

## Installation
In this assignment you will use: 
* os, sys for accessing files
* pdb (optional) for debugging
* NumPy for vectorized operations
* matplotlib for plotting
* mpl_toolkits for 3D plotting
* utils_hw3_coding for helper functions
* PyTorch (for implementing back propagation and neural networks)

In [24]:
import os
import sys
import pdb
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from utils_hw3_coding import *

In [25]:
import torch
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Read in JIGSAWS Data

In [26]:
jigsaws_path = 'JIGSAW/'

Train, Test = read_jigsaws_data(jigsaws_path)
train_data = Train[0]; train_labels = Train[1]; train_files = Train[2]
test_data  = Test[0];  test_labels  = Test[1];  test_files  = Test[2]

## Linear Neural Network

In [27]:
class LinearNet(torch.nn.Module):
    def __init__(self, window_size, hid_dim):        
        super(LinearNet, self).__init__()
        # TODO: fill in the correct linear layer sizes. 
        # Note, you may need to pass in parameter(s) when LinearNet is initialized
        self.L1      = torch.nn.Linear(window_size*6, hid_dim)
        self.L2      = torch.nn.Linear(hid_dim, 1)
        self.ReLU    = torch.nn.ReLU()
        
    def forward(self, x):
        ''' forward pass '''
        # TODO: implement the forward pass
        x = self.ReLU(self.L1(x))
        x = torch.sigmoid(self.L2(x))
        return x
#         pass

    def predict(self, x):
        ''' predict labels 0/1 '''
        # TODO: implement a predict function that predicts 0 or 1 for each example
        x = self.forward(x)
#         pass
        return x

In [28]:
''' Set the Training Parameters '''
batch_size  = 100 
window_size = 100
epochs      = 150
learning_rate = 0.0001
hid_dim     = 150

In [29]:
model = LinearNet(window_size, hid_dim)
# TODO: choose a loss function
loss_fn = nn.BCELoss()
# TODO: choose an optimizer
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)

In [31]:
''' Training Loop  '''
Loss = []
Train_Accuracy = []
Test_Accuracy  = []

for epic in range(epochs):
    model.train()
    window, window_labels = get_window(train_data, train_labels, window_size, batch_size)
    
    ''' ...... '''
    # TODO: implement training on your network, compute the loss, update weights
    window = torch.from_numpy(window).float()
    window_labels = torch.from_numpy(window_labels).float()
    
    optimizer.zero_grad()
    
    out = model(window)
    loss = loss_fn(out, window_labels)
    loss.backward()
    optimizer.step()
    
    ''' ...... '''
    

    Loss.append(loss.detach().item())
    print("Loss: " + str(loss.detach().item()), end='\r')
    print(epic)
    
    if(epic % 10 == 0):
        ''' We have implemented the function compute_accuracy to help you evaluate your network. 
        It returns the accuracy of the binary skill predictions.'''
        train_accuracy, train_predictions = compute_accuracy(train_data, train_labels, model, window_size)
        test_accuracy,  test_predictions  = compute_accuracy(test_data, test_labels, model, window_size)
        Train_Accuracy.append(train_accuracy)
        Test_Accuracy.append(test_accuracy)  
        print(test_accuracy)

0oss: 0.4860273599624634
0.9090909090909091
1oss: 0.5073659420013428
2oss: 0.5196100473403931
3oss: 0.5039699077606201
4oss: 0.4883817732334137
5oss: 0.5210750102996826
6oss: 0.5174717903137207
7oss: 0.4328964650630951
8oss: 0.4629075527191162
9oss: 0.45600631833076477
10ss: 0.519481897354126
0.9090909090909091
11ss: 0.45604920387268066
12ss: 0.47404271364212036
13ss: 0.5161123871803284
14ss: 0.48916545510292053
15ss: 0.49720609188079834
16ss: 0.4163954555988312
17ss: 0.5283619165420532
18ss: 0.4896260201931
19ss: 0.5084235072135925
20ss: 0.41672104597091675
0.9090909090909091
21ss: 0.38872870802879333
22ss: 0.4947454035282135
23ss: 0.4482777714729309
24ss: 0.5077322125434875
25ss: 0.5124091506004333
26ss: 0.4053930938243866
27ss: 0.435245543718338
28ss: 0.4217585623264313
29ss: 0.4081770181655884
30ss: 0.5277518630027771
0.9090909090909091
31ss: 0.4432532489299774
32ss: 0.5160080194473267
33ss: 0.5343470573425293
34ss: 0.49491870403289795
35ss: 0.4469638466835022
36ss: 0.4074164330959

In [20]:
''' Print out the model accuracy after training '''
train_accuracy, train_predictions = compute_accuracy(train_data, train_labels, model, window_size)
test_accuracy,  test_predictions  = compute_accuracy(test_data, test_labels, model, window_size)

print("Training Accuracy: " + str(train_accuracy))
print("Testing Accuracy: " + str(test_accuracy))

Training Accuracy: 0.8571428571428571
Testing Accuracy: 0.9090909090909091


In [1]:
''' Evaluate your Network '''

# plot the loss 
fig = plt.figure()
plt.plot(Loss, label="Loss")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

NameError: name 'plt' is not defined

In [ ]:
''' Visualize Accuracy '''

fig = plt.figure()
plt.plot(Train_Accuracy, label="Train_Accuracy")
plt.plot(Test_Accuracy, label="Test_Accuracy")

plt.ylim([0, 1])
plt.legend()
plt.show()

## Written Questions
* How does the learning rate impact training? Try training networks with 3 different learning rates. Explain the impact you observe of learning rate on training? <br><br>
<i> Your response here. </i>

* How many epochs are required for your network to "converge"? How can you tell if the network has converged? How does the time to convergence relate to the learning rate? Refer back to the 3 different learning rates you tried in the question above. <br><br>
<i> Your response here. </i>

* In your LinearNet, you project the input data into some smaller dimension (`size_2`). Does changing `size_2` change the performance of your network? Explain using 3 different dimensions you tried.